In [1]:
# --- Ensemble Learning Example with Titanic Dataset ---
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score

# Base models
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier

# Ensemble methods
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier, VotingClassifier

In [2]:
# 1. Load Titanic dataset (GitHub version)
url = "https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv"
data = pd.read_csv(url)

print("Dataset Shape:", data.shape)
print(data.head())

# 2. Preprocessing
# Drop unnecessary columns
data = data.drop(["Name", "Ticket", "Cabin"], axis=1)

# Fill missing values
data["Age"].fillna(data["Age"].median(), inplace=True)
data["Embarked"].fillna(data["Embarked"].mode()[0], inplace=True)

Dataset Shape: (891, 12)
   PassengerId  Survived  Pclass  \
0            1         0       3   
1            2         1       1   
2            3         1       3   
3            4         1       1   
4            5         0       3   

                                                Name     Sex   Age  SibSp  \
0                            Braund, Mr. Owen Harris    male  22.0      1   
1  Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0      1   
2                             Heikkinen, Miss. Laina  female  26.0      0   
3       Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.0      1   
4                           Allen, Mr. William Henry    male  35.0      0   

   Parch            Ticket     Fare Cabin Embarked  
0      0         A/5 21171   7.2500   NaN        S  
1      0          PC 17599  71.2833   C85        C  
2      0  STON/O2. 3101282   7.9250   NaN        S  
3      0            113803  53.1000  C123        S  
4      0            373450   8

C:\Users\AIMS TECH\AppData\Local\Temp\ipykernel_4908\1790101521.py:13: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data["Age"].fillna(data["Age"].median(), inplace=True)
C:\Users\AIMS TECH\AppData\Local\Temp\ipykernel_4908\1790101521.py:14: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a 

In [3]:
# Encode categorical features
le = LabelEncoder()
data["Sex"] = le.fit_transform(data["Sex"])
data["Embarked"] = le.fit_transform(data["Embarked"])

In [4]:
# Features and target
X = data.drop("Survived", axis=1)
y = data["Survived"]

In [5]:
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [6]:
# 3. Train individual models
log_clf = LogisticRegression(max_iter=1000)
dt_clf = DecisionTreeClassifier(random_state=42)
knn_clf = KNeighborsClassifier()

models = [("Logistic Regression", log_clf),
          ("Decision Tree", dt_clf),
          ("KNN", knn_clf)]

for name, model in models:
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    print(f"{name} Accuracy: {accuracy_score(y_test, y_pred):.4f}")


Logistic Regression Accuracy: 0.8101
Decision Tree Accuracy: 0.7654
KNN Accuracy: 0.6592


In [7]:
# 4. Ensemble Learning
# Random Forest
rf_clf = RandomForestClassifier(n_estimators=100, random_state=42)
rf_clf.fit(X_train, y_train)
print("Random Forest Accuracy:", accuracy_score(y_test, rf_clf.predict(X_test)))


Random Forest Accuracy: 0.8268156424581006


In [9]:
# Bagging with Decision Tree
bag_clf = BaggingClassifier(
    estimator=DecisionTreeClassifier(),
    n_estimators=100,
    random_state=42
)
bag_clf.fit(X_train, y_train)
print("Bagging Accuracy:", accuracy_score(y_test, bag_clf.predict(X_test)))


Bagging Accuracy: 0.8100558659217877


In [10]:
# Voting Classifier (soft voting)
voting_clf = VotingClassifier(
    estimators=[("lr", log_clf), ("dt", dt_clf), ("knn", knn_clf)],
    voting="soft"
)
voting_clf.fit(X_train, y_train)
print("Voting Classifier Accuracy:", accuracy_score(y_test, voting_clf.predict(X_test)))


Voting Classifier Accuracy: 0.770949720670391
